### 1. Setup + imports

In [27]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

from src.bootcamp_data.config import make_paths
from src.bootcamp_data.io import read_orders_csv, read_users_csv, write_parquet, read_parquet
from src.bootcamp_data.transforms import enforce_schema
import pandas as pd
import sys
import plotly.express as px

path = make_paths(ROOT)

FIGS = ROOT/ Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)




### 2. Read processed data and quick overview

In [28]:

orders_output = path.processed / "analytics_table_x.parquet"
df = pd.read_parquet(orders_output)
df.head(10)

,order_id,user_id,amount,quantity,created_at,status,status_proccess,amount__isna,quantity__isna,date,year,month,dow,hour,quarter,country,signup_date,amount_winsor,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,12.0,Monday,10.0,4.0,SA,2025-11-15 00:00:00+00:00,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,12.0,Monday,11.0,4.0,SA,2025-11-20 00:00:00+00:00,8.0,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,12.0,Tuesday,9.0,4.0,AE,2025-11-22 00:00:00+00:00,<NA>,False
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,12.0,Wednesday,14.0,4.0,SA,2025-11-15 00:00:00+00:00,25.0,False
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,NaN,NaN,None,NaN,NaN,SA,2025-11-25 00:00:00+00:00,100.0,False
5,A0006,0002,15.75,1,2025-12-04 09:10:00+00:00,Paid,paid,False,False,2025-12-04,2025.0,12.0,Thursday,9.0,4.0,SA,2025-11-20 00:00:00+00:00,15.75,False
6,A0007,0003,200.0,2,2025-12-04 12:45:00+00:00,PAID,paid,False,False,2025-12-04,2025.0,12.0,Thursday,12.0,4.0,AE,2025-11-22 00:00:00+00:00,158.0,True
7,A0008,0005,5.0,1,2025-12-05 08:00:00+00:00,paid,paid,False,False,2025-12-05,2025.0,12.0,Friday,8.0,4.0,SA,2025-11-28 00:00:00+00:00,5.0,False
8,A0009,0006,<NA>,3,2025-12-05 10:20:00+00:00,Paid,paid,True,False,2025-12-05,2025.0,12.0,Friday,10.0,4.0,AE,2025-12-01 00:00:00+00:00,<NA>,False
9,A0010,0001,7.25,1,2025-12-05 16:40:00+00:00,refund,refund,False,False,2025-12-05,2025.0,12.0,Friday,16.0,4.0,SA,2025-11-15 00:00:00+00:00,7.25,False


In [122]:
print("rows count", len(df))
print("Data type:", df.dtypes.head(15))
print("Top missing columns", df.isna().sum().sort_values(ascending=False).head(10))

rows count 50
Data type: order_id                string[python]
user_id                 string[python]
amount                         Float64
quantity                         Int64
created_at         datetime64[ns, UTC]
status                          object
status_proccess                 object
amount__isna                      bool
quantity__isna                    bool
date                            object
year                           float64
month                          float64
dow                             object
hour                           float64
quarter                        float64
dtype: object
Top missing columns amount           7
amount_winsor    7
signup_date      7
date             4
dow              4
quantity         4
created_at       4
quarter          4
hour             4
year             4
dtype: int64


In [24]:
df.describe()

,amount,quantity,year,month,hour,quarter,amount_winsor
count,43.0,46.0,46.0,46.0,46.000000,46.0,43.0
mean,39.44907,1.434783,2025.0,12.0,12.152174,4.0,38.521163
std,37.471874,0.65497,0.0,0.0,3.319755,0.0,33.471153
min,0.0,1.0,2025.0,12.0,7.000000,4.0,2.1
25%,13.375,1.0,2025.0,12.0,9.000000,4.0,13.375
50%,25.0,1.0,2025.0,12.0,11.500000,4.0,25.0
75%,57.75,2.0,2025.0,12.0,14.750000,4.0,57.75
max,200.0,3.0,2025.0,12.0,19.000000,4.0,158.0


In [11]:
summary_stats = {
    "time_start": df["created_at"].min(),
    "time_end": df["created_at"].max(),
    "n_orders": len(df),
    "paid_rate": (df["status_proccess"] == "paid").mean(),
    "missing_amount_rate": df["amount__isna"].mean(),
    "missing_quantity_rate": df["quantity__isna"].mean(),
    "amount_outlier_rate": df["amount__is_outlier"].mean(),
}

summary_stats


{'time_start': Timestamp('2025-12-01 10:05:00+0000', tz='UTC'),
 'time_end': Timestamp('2025-12-20 09:00:00+0000', tz='UTC'),
 'n_orders': 50,
 'paid_rate': 0.9,
 'missing_amount_rate': 0.14,
 'missing_quantity_rate': 0.08,
 'amount_outlier_rate': 0.02}

### 3. EDA Questions

- What is the refund rate by country?

In [29]:
status_by_country = df.groupby(['country', 'status_proccess']).size().reset_index(name='count')

refund_rate = df[df['status_proccess'] == 'refund'].groupby('country').size() / df.groupby('country').size() * 100

print("Refund rate by country:")
print(refund_rate)

fig = px.bar(status_by_country, x='country', y='count', color='status_proccess', text='count', title='Order status distribution by country',)
fig.update_yaxes(title_text="Number of orders")
fig.show()

save_fig(fig, FIGS / "refund_by_country.png")



Refund rate by country:
country
AE    15.384615
KW    12.500000
SA     6.896552
dtype: float64


- who are the top 10 Users by Revenue?

In [30]:
user_revenue = (df.groupby("user_id")["amount"].sum().reset_index())

fig = px.bar(user_revenue.head(10),x="user_id",y="amount",title="Top 10 users by revenue")
fig.show()

save_fig(fig, FIGS / "top_users_by_revenue.png")



Resorting to unclean kill browser.


- Which countries has the higest revenue?


In [31]:
revenue = (
    df.groupby("country", dropna=False)
    .agg( n_orders=("order_id","size"), revenue=("amount","sum"))
    .reset_index().sort_values("revenue", ascending=False)
)

fig = px.bar(revenue, x="country", y="revenue", title="Revenue by country")
save_fig(fig, FIGS / "revenue_by_country.png")
fig



- which day has highest revenue?

In [32]:

dow_revenue = (df.groupby("dow")["amount"].sum().reset_index())

fig = px.bar(dow_revenue, x="dow", y="amount", title="Total revenue by day of week")
fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Revenue")

fig.show()

save_fig(fig, FIGS / "revenue_by_days.png")


Resorting to unclean kill browser.


- which days has many orders volume and days has less?

In [33]:
daily_summary = df.groupby('dow').agg(order_count=('order_id', 'count'),total_amount=('amount', 'sum') ).reset_index()

fig = px.bar(daily_summary, x="dow", y="order_count", title="Order volume by day of week")
fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Number of orders")

fig.show()

save_fig(fig, FIGS / "orders_by_days.png")


- which hours has many orders volume and hours has less?

In [14]:
hourly = df.groupby('hour').agg(order_count=('order_id', 'count')).reset_index()

fig = px.line(hourly, x='hour', y='order_count', title='Order volume by hour')
fig.update_xaxes(title_text="Hours")
fig.update_yaxes(title_text="Number of orders")
fig.show()

save_fig(fig, FIGS / "orders_by_hours.png")




Resorting to unclean kill browser.


- What is the distribution of order amounts and are there any outliers?

In [18]:
outliers_count = df['amount__is_outlier'].sum()
print(f"\nNumber of outliers: {outliers_count}")
print(f"Percentage of outliers: {outliers_count/len(df)*100:.2f}%")

fig = px.histogram(df, x="amount", nbins=30, title="Amount distribution")
fig.update_xaxes(title_text="Amount")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_outlier.png")
fig


Number of outliers: 1
Percentage of outliers: 2.00%


- Do refunds happen more at specific times of day? 


In [16]:
refunds_by_hour = (df[df["status_proccess"] == "refund"].groupby("hour").size().reset_index(name="refund_count"))

fig = px.bar(refunds_by_hour,x="hour",y="refund_count",title="Refunds by hour of day")
fig.show()


- Does missing data spike at specific hours?

In [17]:
missing_by_hour = (df.groupby("hour")[["amount__isna", "quantity__isna"]].sum().reset_index())

fig = px.line(missing_by_hour, x="hour", y=["amount__isna", "quantity__isna"], title="Missing data rate by hour")
fig.show()


### 4. Bootstrap comparison

In [38]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

d = df.assign(is_refund=df["status_proccess"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]
print("n_SA:", len(a), "n_AE:", len(b))
res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)

n_SA: 29 n_AE: 13
{'diff_mean': -0.08488063660477455, 'ci_low': -0.3158488063660478, 'ci_high': 0.10344827586206896}
